In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import re
import os
import time
from IterativeLIFModel import *
from SCNNLayer3Conv import *
from DVS_CIFAR10 import DVS_CIFAR10

In [2]:
# Dacay learning_rate
def lr_scheduler(optimizer, epoch, init_lr=0.1, lr_decay_epoch=50):
    """Decay learning rate by a factor of 0.1 every lr_decay_epoch epochs."""
    if epoch % lr_decay_epoch == 0 and epoch > 1:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1
    return optimizer

In [3]:
def main():
    snn = SCNN()
    snn.to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(snn.parameters(), lr=learning_rate)
    
    for epoch in range(num_epochs):
        train_loss = 0
        start_time = time.time()
        for i, (train_images, train_labels) in enumerate(trainLoader):
            snn.zero_grad()
            optimizer.zero_grad()

            train_images = train_images.float().to(device)
            train_predicts = snn(train_images)
            train_labels = torch.zeros(train_labels.shape[0], num_classes).scatter_(1, train_labels.view(-1,1), 1.)
                                                #将index视作列（dim = 1），按照train_labels.view(-1,1)值作为index，把src =“1”插入到zeros中
            loss = criterion(train_predicts.cpu(), train_labels)
            train_loss = loss.item() + train_loss
            loss.backward()
            optimizer.step()

            if (i + 1) % 100 == 0:
                print('Epoch [%d/%d], Train Step [%d/%d], Train Loss = %.5f' 
                        %(epoch + 1, num_epochs, i + 1, len(trainSet)//batch_size, train_loss))
                train_loss = 0
                print('Time elasped:', time.time() - start_time)
        correct = 0
        total = 0
        optimizer = lr_scheduler(optimizer, epoch, learning_rate, 40)

        with torch.no_grad():
            for j, (test_images, test_targets) in enumerate(testLoader):
                optimizer.zero_grad()
                snn.zero_grad()

                test_images = test_images.float().to(device)
                test_predicts = snn(test_images)
                test_labels = torch.zeros(test_targets.shape[0], num_classes).scatter_(1, test_targets.view(-1,1), 1.)
                loss = criterion(test_predicts.cpu(), test_labels)

                _, predicted = test_predicts.cpu().max(1)
                total = total + float(test_targets.size(0))
                correct = correct + float(predicted.eq(test_targets).sum().item())
                if (j + 1) % 25 == 0:
                    acc = 100. * float(correct)/float(total)
                    print('Test Step [%d/%d], Acc: %.5f' %(j + 1, len(testLoader), acc))
        acc = float(100 * correct / total)
        print('Epoch [%d/%d] \t Test Accuracy Over Test Dataset: %.3f' %(epoch + 1, num_epochs, acc))
        acc_record.append(acc)
        if epoch % 5 == 0:
            print('Saving......')
            state = {
                'net': snn.state_dict(),
                'acc': acc,
                'epoch': epoch,
                'acc_record': acc_record,
            }
            if not os.path.isdir('./history'):
                os.mkdir('./history')
            torch.save(state, './history/' + names + '.pk')

In [4]:
if __name__=="__main__":
    dataPath = '../../../DATA/DVS-CIFAR10/processed/'
    start_epoch = 0  # start from epoch 0 or last checkpoint epoch
    acc_record = []
    num_epochs = 100 # max epoch
    num_classes = 10
    batch_size = 5
    names = 'STBPmodelDVS-CIFAR10'
    learning_rate = 1e-3
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    trainSet = DVS_CIFAR10(dataPath = dataPath, mode = 'train')
    trainLoader = DataLoader(dataset = trainSet, batch_size = 5, shuffle=True, num_workers = 4)
    
    testSet = DVS_CIFAR10(dataPath = dataPath, mode = 'test')
    testLoader = DataLoader(dataset = testSet, batch_size = 5, shuffle=True, num_workers = 4)
    
    main()

Epoch [1/100], Train Step [100/1800], Train Loss = 10.00000
Time elasped: 27.874141931533813
Epoch [1/100], Train Step [200/1800], Train Loss = 10.00000
Time elasped: 51.53021740913391
Epoch [1/100], Train Step [300/1800], Train Loss = 10.00000
Time elasped: 75.1822943687439
Epoch [1/100], Train Step [400/1800], Train Loss = 10.22014
Time elasped: 98.93021821975708
Epoch [1/100], Train Step [500/1800], Train Loss = 10.71928
Time elasped: 122.70219302177429
Epoch [1/100], Train Step [600/1800], Train Loss = 10.73258
Time elasped: 146.5602581501007
Epoch [1/100], Train Step [700/1800], Train Loss = 10.27766
Time elasped: 170.48298811912537
Epoch [1/100], Train Step [800/1800], Train Loss = 10.60013
Time elasped: 194.38738179206848
Epoch [1/100], Train Step [900/1800], Train Loss = 10.33933
Time elasped: 218.40511536598206
Epoch [1/100], Train Step [1000/1800], Train Loss = 10.60095
Time elasped: 242.25528478622437
Epoch [1/100], Train Step [1100/1800], Train Loss = 10.34798
Time elasped:

Epoch [5/100], Train Step [400/1800], Train Loss = 9.15329
Time elasped: 97.34576463699341
Epoch [5/100], Train Step [500/1800], Train Loss = 9.05654
Time elasped: 121.12957310676575
Epoch [5/100], Train Step [600/1800], Train Loss = 9.13790
Time elasped: 144.7406792640686
Epoch [5/100], Train Step [700/1800], Train Loss = 9.23686
Time elasped: 168.4961919784546
Epoch [5/100], Train Step [800/1800], Train Loss = 9.09315
Time elasped: 192.26330018043518
Epoch [5/100], Train Step [900/1800], Train Loss = 9.16911
Time elasped: 215.94055581092834
Epoch [5/100], Train Step [1000/1800], Train Loss = 9.11849
Time elasped: 239.71002769470215
Epoch [5/100], Train Step [1100/1800], Train Loss = 9.10730
Time elasped: 263.3921592235565
Epoch [5/100], Train Step [1200/1800], Train Loss = 9.13115
Time elasped: 287.0501654148102
Epoch [5/100], Train Step [1300/1800], Train Loss = 9.07233
Time elasped: 310.7666599750519
Epoch [5/100], Train Step [1400/1800], Train Loss = 9.14658
Time elasped: 334.4848

Epoch [9/100], Train Step [700/1800], Train Loss = 8.99145
Time elasped: 168.08606576919556
Epoch [9/100], Train Step [800/1800], Train Loss = 9.03309
Time elasped: 191.91810083389282
Epoch [9/100], Train Step [900/1800], Train Loss = 9.01739
Time elasped: 215.68654465675354
Epoch [9/100], Train Step [1000/1800], Train Loss = 8.95920
Time elasped: 240.1772644519806
Epoch [9/100], Train Step [1100/1800], Train Loss = 9.02113
Time elasped: 264.0847957134247
Epoch [9/100], Train Step [1200/1800], Train Loss = 9.19469
Time elasped: 288.03151202201843
Epoch [9/100], Train Step [1300/1800], Train Loss = 9.04186
Time elasped: 312.00138807296753
Epoch [9/100], Train Step [1400/1800], Train Loss = 9.02271
Time elasped: 335.811336517334
Epoch [9/100], Train Step [1500/1800], Train Loss = 8.94513
Time elasped: 359.531533241272
Epoch [9/100], Train Step [1600/1800], Train Loss = 8.94531
Time elasped: 383.4516808986664
Epoch [9/100], Train Step [1700/1800], Train Loss = 8.97249
Time elasped: 407.28

Epoch [13/100], Train Step [900/1800], Train Loss = 8.54351
Time elasped: 216.08736896514893
Epoch [13/100], Train Step [1000/1800], Train Loss = 8.37080
Time elasped: 239.99751591682434
Epoch [13/100], Train Step [1100/1800], Train Loss = 8.41018
Time elasped: 263.85928225517273
Epoch [13/100], Train Step [1200/1800], Train Loss = 8.29539
Time elasped: 287.68335151672363
Epoch [13/100], Train Step [1300/1800], Train Loss = 8.35373
Time elasped: 311.3773136138916
Epoch [13/100], Train Step [1400/1800], Train Loss = 8.38735
Time elasped: 335.12436294555664
Epoch [13/100], Train Step [1500/1800], Train Loss = 8.45323
Time elasped: 358.82721638679504
Epoch [13/100], Train Step [1600/1800], Train Loss = 8.37653
Time elasped: 382.5900068283081
Epoch [13/100], Train Step [1700/1800], Train Loss = 8.27481
Time elasped: 406.367080450058
Epoch [13/100], Train Step [1800/1800], Train Loss = 8.37979
Time elasped: 430.07087659835815
Test Step [25/200], Acc: 16.00000
Test Step [50/200], Acc: 14.400

Epoch [17/100], Train Step [1100/1800], Train Loss = 10.00000
Time elasped: 264.8368594646454
Epoch [17/100], Train Step [1200/1800], Train Loss = 10.00000
Time elasped: 288.74720001220703
Epoch [17/100], Train Step [1300/1800], Train Loss = 10.00000
Time elasped: 312.7710928916931
Epoch [17/100], Train Step [1400/1800], Train Loss = 10.00000
Time elasped: 336.778373003006
Epoch [17/100], Train Step [1500/1800], Train Loss = 10.00000
Time elasped: 360.73342871665955
Epoch [17/100], Train Step [1600/1800], Train Loss = 10.00000
Time elasped: 384.7338058948517
Epoch [17/100], Train Step [1700/1800], Train Loss = 10.00000
Time elasped: 408.75173926353455
Epoch [17/100], Train Step [1800/1800], Train Loss = 10.00000
Time elasped: 432.5783224105835
Test Step [25/200], Acc: 12.80000
Test Step [50/200], Acc: 12.40000
Test Step [75/200], Acc: 10.93333
Test Step [100/200], Acc: 11.20000
Test Step [125/200], Acc: 11.20000
Test Step [150/200], Acc: 10.93333
Test Step [175/200], Acc: 10.62857
Test

Epoch [21/100], Train Step [1300/1800], Train Loss = 10.00000
Time elasped: 313.4698739051819
Epoch [21/100], Train Step [1400/1800], Train Loss = 10.00000
Time elasped: 337.50743794441223
Epoch [21/100], Train Step [1500/1800], Train Loss = 10.00000
Time elasped: 361.5739369392395
Epoch [21/100], Train Step [1600/1800], Train Loss = 10.00000
Time elasped: 385.70686507225037
Epoch [21/100], Train Step [1700/1800], Train Loss = 10.00000
Time elasped: 409.5729537010193
Epoch [21/100], Train Step [1800/1800], Train Loss = 10.00000
Time elasped: 433.72685623168945
Test Step [25/200], Acc: 7.20000
Test Step [50/200], Acc: 10.80000
Test Step [75/200], Acc: 10.66667
Test Step [100/200], Acc: 9.60000
Test Step [125/200], Acc: 9.12000
Test Step [150/200], Acc: 9.46667
Test Step [175/200], Acc: 9.71429
Test Step [200/200], Acc: 10.00000
Epoch [21/100] 	 Test Accuracy Over Test Dataset: 10.000
Saving......
Epoch [22/100], Train Step [100/1800], Train Loss = 10.00000
Time elasped: 26.1843812465667

Epoch [25/100], Train Step [1500/1800], Train Loss = 10.00000
Time elasped: 362.27279114723206
Epoch [25/100], Train Step [1600/1800], Train Loss = 10.00000
Time elasped: 386.27994108200073
Epoch [25/100], Train Step [1700/1800], Train Loss = 10.00000
Time elasped: 410.23522186279297
Epoch [25/100], Train Step [1800/1800], Train Loss = 10.00000
Time elasped: 434.16448974609375
Test Step [25/200], Acc: 7.20000
Test Step [50/200], Acc: 8.40000
Test Step [75/200], Acc: 7.20000
Test Step [100/200], Acc: 7.40000
Test Step [125/200], Acc: 7.52000
Test Step [150/200], Acc: 8.80000
Test Step [175/200], Acc: 8.80000
Test Step [200/200], Acc: 10.00000
Epoch [25/100] 	 Test Accuracy Over Test Dataset: 10.000
Epoch [26/100], Train Step [100/1800], Train Loss = 10.00000
Time elasped: 26.057013511657715
Epoch [26/100], Train Step [200/1800], Train Loss = 10.00000
Time elasped: 49.95508122444153
Epoch [26/100], Train Step [300/1800], Train Loss = 10.00000
Time elasped: 73.83519124984741
Epoch [26/100

KeyboardInterrupt: 

In [5]:

device

device(type='cuda')

In [11]:
for epoch in range(num_epochs):
    train_loss = 0
    start_time = time.time()
    for i, (train_images, train_labels) in enumerate(trainLoader):
        snn.zero_grad()
        optimizer.zero_grad()
        
        train_images = train_images.float().to(device)
        train_predicts = snn(train_images)
        train_labels = torch.zeros(train_labels.shape[0], num_classes).scatter_(1, train_labels.view(-1,1), 1.)
                                            #将index视作列（dim = 1），按照train_labels.view(-1,1)值作为index，把src =“1”插入到zeros中
        loss = criterion(train_predicts.cpu(), train_labels)
        train_loss = loss.item() + train_loss
        loss.backward()
        optimizer.step()
        
        if (i + 1) % 100 == 0:
            print('Epoch [%d/%d], Train Step [%d/%d], Train Loss = %.5f' 
                    %(epoch + 1, num_epochs, i + 1, len(trainSet)//batch_size, train_loss))
            train_loss = 0
            print('Time elasped:', time.time() - start_time)
    correct = 0
    total = 0
    optimizer = lr_scheduler(optimizer, epoch, learning_rate, 40)
    
    with torch.no_grad():
        for j, (test_images, test_targets) in enumerate(testLoader):
            optimizer.zero_grad()
            snn.zero_grad()
            
            test_images = test_images.float().to(device)
            test_predicts = snn(test_images)
            test_labels = torch.zeros(test_targets.shape[0], num_classes).scatter_(1, test_targets.view(-1,1), 1.)
            loss = criterion(test_predicts.cpu(), test_labels)
            
            _, predicted = test_predicts.cpu().max(1)
            total = total + float(test_targets.size(0))
            correct = correct + float(predicted.eq(test_targets).sum().item())
            if (j + 1) % 25 == 0:
                acc = 100. * float(correct)/float(total)
                print('Test Step [%d/%d], Acc: %.5f' %(j + 1, len(testLoader), acc))
    acc = float(100 * correct / total)
    print('Epoch [%d/%d] \t Test Accuracy Over Test Dataset: %.3f' %(epoch + 1, num_epochs, acc))
    acc_record.append(acc)
    if epoch % 5 == 0:
        print('Saving......')
        state = {
            'net': snn.state_dict(),
            'acc': acc,
            'epoch': epoch,
            'acc_record': acc_record,
        }
        if not os.path.isdir('./history'):
            os.mkdir('./history')
        torch.save(state, './history/' + names + '.pk')

Epoch [1/100], Train Step [100/1800], Train Loss = 10.00000
Time elasped: 25.53849148750305
Epoch [1/100], Train Step [200/1800], Train Loss = 10.00000
Time elasped: 49.59444856643677
Epoch [1/100], Train Step [300/1800], Train Loss = 10.11840
Time elasped: 73.76645517349243
Epoch [1/100], Train Step [400/1800], Train Loss = 10.23865
Time elasped: 98.49698448181152
Epoch [1/100], Train Step [500/1800], Train Loss = 10.78983
Time elasped: 124.02805018424988
Epoch [1/100], Train Step [600/1800], Train Loss = 10.53210
Time elasped: 149.21956849098206
Epoch [1/100], Train Step [700/1800], Train Loss = 10.45710
Time elasped: 174.63883090019226
Epoch [1/100], Train Step [800/1800], Train Loss = 10.19314
Time elasped: 199.62103009223938
Epoch [1/100], Train Step [900/1800], Train Loss = 10.01538
Time elasped: 224.41712260246277


KeyboardInterrupt: 